In [5]:
import os
import json
import shutil
import hashlib
from pathlib import Path
from typing import Dict, List, Set
import re
from datetime import datetime

class IncrementalSongConsolidator:
    def __init__(self, base_songs_folder: str, output_folder: str = "consolidated_songs"):
        self.base_songs_folder = Path(base_songs_folder)
        self.output_folder = Path(output_folder)
        self.songs_output = self.output_folder / "songs"
        self.metadata_output = self.output_folder / "metadata"
        
        # Create output directories
        self.songs_output.mkdir(parents=True, exist_ok=True)
        self.metadata_output.mkdir(parents=True, exist_ok=True)
        
        # Data structures to track songs and playlists
        self.unique_songs = {}  # song_id -> song_info (changed from track_uri as key)
        self.playlist_data = {}  # playlist_name -> playlist_info
        self.song_to_playlists = {}  # song_id -> list of playlist names
        
        # Track processed playlists to avoid reprocessing
        self.processed_playlists = set()
        
        # Lookup tables for efficient searching
        self.uri_to_song_id = {}  # track_uri -> song_id
        self.name_artist_to_song_id = {}  # normalized_name_artist -> song_id
        
        # Statistics
        self.stats = {
            'new_songs_added': 0,
            'new_playlists_added': 0,
            'duplicates_skipped': 0,
            'playlists_updated': 0,
            'existing_songs_found': 0,
            'songs_added_to_existing_playlists': 0
        }
        
        # Load existing metadata if available
        self.load_existing_metadata()
    
    def load_existing_metadata(self):
        """Load existing metadata from previous runs and preserve ALL data"""
        print("🔍 Loading existing metadata...")
        
        songs_db_path = self.metadata_output / 'songs_database.json'
        playlists_db_path = self.metadata_output / 'playlists_database.json'
        mapping_db_path = self.metadata_output / 'song_playlist_mapping.json'
        
        # Load songs database
        if songs_db_path.exists():
            try:
                with open(songs_db_path, 'r', encoding='utf-8') as f:
                    songs_data = json.load(f)
                    
                # Load ALL existing songs - use song_id as primary key
                existing_songs = songs_data.get('songs', {})
                for song_id, song_info in existing_songs.items():
                    self.unique_songs[song_id] = song_info
                    
                    # Build lookup tables for efficient duplicate detection
                    metadata = song_info.get('metadata', {})
                    track_uri = metadata.get('track_uri', '')
                    if track_uri:
                        self.uri_to_song_id[track_uri] = song_id
                    
                    # Create name+artist lookup
                    track_name = metadata.get('track_name', '').lower().strip()
                    artists = metadata.get('artists_string', '').lower().strip()
                    if track_name and artists:
                        key = f"{track_name}|{artists}"
                        self.name_artist_to_song_id[key] = song_id
                    
                    self.stats['existing_songs_found'] += 1
                
                print(f"   ✅ Loaded {len(self.unique_songs)} existing songs")
            except Exception as e:
                print(f"   ❌ Error loading songs database: {e}")
        
        # Load playlists database  
        if playlists_db_path.exists():
            try:
                with open(playlists_db_path, 'r', encoding='utf-8') as f:
                    playlists_data = json.load(f)
                    existing_playlists = playlists_data.get('playlists', {})
                    
                    # Preserve ALL existing playlist data
                    for playlist_name, playlist_info in existing_playlists.items():
                        self.playlist_data[playlist_name] = playlist_info
                        # Don't mark as processed yet - let the timestamp check decide
                
                print(f"   ✅ Loaded {len(self.playlist_data)} existing playlists")
            except Exception as e:
                print(f"   ❌ Error loading playlists database: {e}")
        
        # Load song-to-playlist mapping
        if mapping_db_path.exists():
            try:
                with open(mapping_db_path, 'r', encoding='utf-8') as f:
                    mapping_data = json.load(f)
                    existing_mappings = mapping_data.get('song_to_playlists', {})
                    
                    # Preserve ALL existing mappings
                    for song_id, playlists in existing_mappings.items():
                        self.song_to_playlists[song_id] = playlists.copy()
                
                print(f"   ✅ Loaded mappings for {len(self.song_to_playlists)} songs")
            except Exception as e:
                print(f"   ❌ Error loading song-playlist mapping: {e}")
        
        # Verify data consistency
        total_loaded = len(self.unique_songs) + len(self.playlist_data) + len(self.song_to_playlists)
        
        if total_loaded > 0:
            print(f"📊 Loaded totals:")
            print(f"   • Songs: {len(self.unique_songs)}")
            print(f"   • Playlists: {len(self.playlist_data)}")  
            print(f"   • Mappings: {len(self.song_to_playlists)}")
            print("✅ Found existing metadata - will merge new data with existing data")
        else:
            print("🆕 No existing metadata found - starting fresh")
    
    def generate_song_id(self, track_name: str, artists: str) -> str:
        """Generate a unique ID for a song based on track name and artists"""
        # Clean the string for ID generation
        clean_string = f"{track_name}_{artists}".lower()
        clean_string = re.sub(r'[^a-z0-9_]', '', clean_string)
        
        # Generate a hash for uniqueness
        hash_object = hashlib.md5(clean_string.encode())
        return f"song_{hash_object.hexdigest()[:12]}"
    
    def find_existing_song_id(self, metadata: dict) -> str:
        """Find existing song ID if song already exists, otherwise return None"""
        track_uri = metadata.get('track_uri', '')
        track_name = metadata.get('track_name', '').lower().strip()
        artists = metadata.get('artists_string', '').lower().strip()
        
        # First check by URI (most reliable)
        if track_uri and track_uri in self.uri_to_song_id:
            return self.uri_to_song_id[track_uri]
        
        # Then check by name + artists
        if track_name and artists:
            key = f"{track_name}|{artists}"
            if key in self.name_artist_to_song_id:
                return self.name_artist_to_song_id[key]
        
        return None
    
    def normalize_filename(self, filename: str) -> str:
        """Normalize filename for comparison"""
        return filename.lower().replace('-', '').replace('_', '').replace(' ', '')
    
    def find_song_files(self, playlist_folder: Path) -> List[Path]:
        """Find all audio files in a playlist folder"""
        audio_extensions = {'.mp3', '.webm', '.wav', '.flac', '.m4a'}
        song_files = []
        
        for file_path in playlist_folder.iterdir():
            if file_path.is_file() and file_path.suffix.lower() in audio_extensions:
                song_files.append(file_path)
        
        return song_files
    
    def playlist_needs_update(self, playlist_folder: Path, playlist_name: str) -> bool:
        """Check if playlist needs to be updated based on modification time or new songs"""
        json_file = playlist_folder / "enhanced_download_summary.json"
        if not json_file.exists():
            return False
        
        # Always process if playlist is new
        if playlist_name not in self.processed_playlists:
            return True
        
        # Check if the enhanced_download_summary.json has been modified
        existing_playlist = self.playlist_data.get(playlist_name, {})
        existing_timestamp = existing_playlist.get('timestamp', '')
        
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            current_timestamp = data.get('download_info', {}).get('timestamp', '')
            
            # If timestamps differ, playlist needs update
            return current_timestamp != existing_timestamp
        except:
            return True
    
    def get_existing_filename(self, song_id: str) -> str:
        """Get the existing filename for a song if it exists"""
        if song_id in self.unique_songs:
            return self.unique_songs[song_id].get('filename', '')
        return ''
    
    def process_playlist_folder(self, playlist_folder: Path):
        """Process a single playlist folder"""
        playlist_name = playlist_folder.name
        json_file = playlist_folder / "enhanced_download_summary.json"
        
        # Skip folders without the JSON file
        if not json_file.exists():
            print(f"⏭️  Skipping {playlist_name}: No enhanced_download_summary.json found")
            return
        
        # Check if playlist needs update
        if not self.playlist_needs_update(playlist_folder, playlist_name):
            print(f"⏭️  Skipping {playlist_name}: Already up to date")
            return
        
        is_new_playlist = playlist_name not in self.processed_playlists
        action = "🆕 Processing new playlist" if is_new_playlist else "🔄 Updating playlist"
        print(f"{action}: {playlist_name}")
        
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except Exception as e:
            print(f"❌ Error reading JSON from {playlist_name}: {e}")
            return
        
        # Update playlist metadata (merge with existing if present)
        download_info = data.get('download_info', {})
        
        if playlist_name in self.playlist_data:
            # Update existing playlist data
            existing_playlist = self.playlist_data[playlist_name]
            existing_playlist.update({
                'total_tracks': download_info.get('total_tracks', existing_playlist.get('total_tracks', 0)),
                'successful_downloads': download_info.get('successful_downloads', existing_playlist.get('successful_downloads', 0)),
                'source_url': download_info.get('source_url', existing_playlist.get('source_url', '')),
                'timestamp': download_info.get('timestamp', existing_playlist.get('timestamp', '')),
                'folder_path': str(playlist_folder),
                'last_updated': datetime.now().isoformat()
            })
        else:
            # Create new playlist entry
            self.playlist_data[playlist_name] = {
                'name': playlist_name,
                'total_tracks': download_info.get('total_tracks', 0),
                'successful_downloads': download_info.get('successful_downloads', 0),
                'source_url': download_info.get('source_url', ''),
                'timestamp': download_info.get('timestamp', ''),
                'folder_path': str(playlist_folder),
                'last_updated': datetime.now().isoformat(),
                'created_at': datetime.now().isoformat()
            }
        
        if is_new_playlist:
            self.stats['new_playlists_added'] += 1
        else:
            self.stats['playlists_updated'] += 1
        
        # Get all song files in the folder
        song_files = self.find_song_files(playlist_folder)
        
        # Process each song in the download results
        download_results = data.get('download_results', [])
        
        for result in download_results:
            if result.get('status') != 'success':
                continue
                
            metadata = result.get('metadata', {})
            filename = result.get('filename', '')
            
            # Find the actual file
            matching_file = None
            for song_file in song_files:
                if self.normalize_filename(song_file.name) == self.normalize_filename(filename):
                    matching_file = song_file
                    break
            
            if not matching_file:
                print(f"⚠️  Warning: Could not find file {filename} in {playlist_name}")
                continue
            
            # Check if this song already exists
            existing_song_id = self.find_existing_song_id(metadata)
            
            if existing_song_id:
                # Song exists - add playlist to existing song
                existing_song = self.unique_songs[existing_song_id]
                
                # Add playlist to song if not already there
                if playlist_name not in existing_song.get('playlists', []):
                    existing_song['playlists'].append(playlist_name)
                    existing_song['last_updated'] = datetime.now().isoformat()
                    self.stats['songs_added_to_existing_playlists'] += 1
                
                # Update song-to-playlist mapping
                if existing_song_id not in self.song_to_playlists:
                    self.song_to_playlists[existing_song_id] = []
                if playlist_name not in self.song_to_playlists[existing_song_id]:
                    self.song_to_playlists[existing_song_id].append(playlist_name)
                
                self.stats['duplicates_skipped'] += 1
                print(f"   📎 Duplicate found: {filename} (existing: {existing_song_id})")
                
            else:
                # New song - create entry
                song_id = self.generate_song_id(
                    metadata.get('track_name', ''),
                    metadata.get('artists_string', '')
                )
                
                # Determine filename - use existing if song_id already exists, otherwise create new
                existing_filename = self.get_existing_filename(song_id)
                if existing_filename:
                    new_filename = existing_filename
                else:
                    file_extension = matching_file.suffix
                    new_filename = f"{song_id}{file_extension}"
                
                new_file_path = self.songs_output / new_filename
                
                # Only copy if file doesn't exist
                if not new_file_path.exists():
                    try:
                        shutil.copy2(matching_file, new_file_path)
                        print(f"   ✅ Copied: {filename} -> {new_filename}")
                    except Exception as e:
                        print(f"   ❌ Error copying {filename}: {e}")
                        continue
                else:
                    print(f"   📁 File already exists: {new_filename}")
                
                # Store/update song info
                if song_id in self.unique_songs:
                    # Update existing song entry
                    existing_song = self.unique_songs[song_id]
                    if playlist_name not in existing_song.get('playlists', []):
                        existing_song['playlists'].append(playlist_name)
                        existing_song['last_updated'] = datetime.now().isoformat()
                else:
                    # Create new song entry
                    self.unique_songs[song_id] = {
                        'song_id': song_id,
                        'filename': new_filename,
                        'original_filename': filename,
                        'file_path': str(new_file_path),
                        'metadata': metadata,
                        'playlists': [playlist_name],
                        'added_at': datetime.now().isoformat(),
                        'last_updated': datetime.now().isoformat()
                    }
                    
                    # Update lookup tables
                    track_uri = metadata.get('track_uri', '')
                    if track_uri:
                        self.uri_to_song_id[track_uri] = song_id
                    
                    track_name = metadata.get('track_name', '').lower().strip()
                    artists = metadata.get('artists_string', '').lower().strip()
                    if track_name and artists:
                        key = f"{track_name}|{artists}"
                        self.name_artist_to_song_id[key] = song_id
                    
                    self.stats['new_songs_added'] += 1
                
                # Update song-to-playlist mapping
                if song_id not in self.song_to_playlists:
                    self.song_to_playlists[song_id] = []
                if playlist_name not in self.song_to_playlists[song_id]:
                    self.song_to_playlists[song_id].append(playlist_name)
        
        # Mark playlist as processed
        self.processed_playlists.add(playlist_name)
    
    def cleanup_orphaned_data(self):
        """Remove data for playlists that no longer exist"""
        current_playlist_folders = {f.name for f in self.base_songs_folder.iterdir() if f.is_dir()}
        
        # Find playlists in metadata that no longer exist
        orphaned_playlists = set(self.playlist_data.keys()) - current_playlist_folders
        
        if orphaned_playlists:
            print(f"\n🗑️  Found {len(orphaned_playlists)} orphaned playlists: {orphaned_playlists}")
            
            # Remove orphaned playlists
            for playlist_name in orphaned_playlists:
                del self.playlist_data[playlist_name]
                self.processed_playlists.discard(playlist_name)
            
            # Clean up song-to-playlist mappings
            for song_id in list(self.song_to_playlists.keys()):
                # Remove orphaned playlists from song mappings
                original_playlists = self.song_to_playlists[song_id].copy()
                self.song_to_playlists[song_id] = [
                    p for p in self.song_to_playlists[song_id] 
                    if p not in orphaned_playlists
                ]
                
                # Remove songs that are no longer in any playlist
                if not self.song_to_playlists[song_id]:
                    del self.song_to_playlists[song_id]
            
            # Update unique_songs playlist lists
            for song_id in list(self.unique_songs.keys()):
                original_playlists = self.unique_songs[song_id]['playlists'].copy()
                self.unique_songs[song_id]['playlists'] = [
                    p for p in self.unique_songs[song_id]['playlists']
                    if p not in orphaned_playlists
                ]
                
                # Remove songs that are no longer in any playlist
                if not self.unique_songs[song_id]['playlists']:
                    # Remove the physical file
                    song_file = Path(self.unique_songs[song_id]['file_path'])
                    if song_file.exists():
                        try:
                            song_file.unlink()
                            print(f"   🗑️  Removed orphaned song file: {song_file.name}")
                        except Exception as e:
                            print(f"   ❌ Error removing {song_file.name}: {e}")
                    
                    # Remove from lookup tables
                    metadata = self.unique_songs[song_id].get('metadata', {})
                    track_uri = metadata.get('track_uri', '')
                    if track_uri in self.uri_to_song_id:
                        del self.uri_to_song_id[track_uri]
                    
                    track_name = metadata.get('track_name', '').lower().strip()
                    artists = metadata.get('artists_string', '').lower().strip()
                    if track_name and artists:
                        key = f"{track_name}|{artists}"
                        if key in self.name_artist_to_song_id:
                            del self.name_artist_to_song_id[key]
                    
                    del self.unique_songs[song_id]
    
    def save_metadata(self):
        """Save consolidated metadata to JSON files - MERGE with existing data"""
        print("\n💾 Saving metadata...")
        
        # 1. SONGS DATABASE - Load existing and merge
        songs_db_path = self.metadata_output / 'songs_database.json'
        existing_songs_db = {'songs': {}, 'stats': {}}
        
        if songs_db_path.exists():
            try:
                with open(songs_db_path, 'r', encoding='utf-8') as f:
                    existing_songs_db = json.load(f)
                print(f"   📖 Loaded existing songs database with {len(existing_songs_db.get('songs', {}))} songs")
            except Exception as e:
                print(f"   ⚠️  Warning: Could not load existing songs database: {e}")
        
        # Merge all songs (existing + current)
        all_songs = existing_songs_db.get('songs', {}).copy()
        
        # Add/update songs from current session
        for song_id, song_info in self.unique_songs.items():
            all_songs[song_id] = song_info
        
        # Prepare stats with history
        existing_stats = existing_songs_db.get('stats', {})
        new_stats = {
            'total_unique_songs': len(all_songs),
            'total_playlists': len(self.playlist_data),
            'generated_at': datetime.now().isoformat(),
            'last_session_stats': self.stats.copy()
        }
        
        # Preserve historical info
        if 'first_created_at' in existing_stats:
            new_stats['first_created_at'] = existing_stats['first_created_at']
        else:
            new_stats['first_created_at'] = datetime.now().isoformat()
        
        if 'session_history' in existing_stats:
            new_stats['session_history'] = existing_stats['session_history'].copy()
        else:
            new_stats['session_history'] = []
        
        # Add current session to history
        new_stats['session_history'].append({
            'timestamp': datetime.now().isoformat(),
            'stats': self.stats.copy()
        })
        
        # Save songs database
        songs_db = {
            'songs': all_songs,
            'stats': new_stats
        }
        
        with open(songs_db_path, 'w', encoding='utf-8') as f:
            json.dump(songs_db, f, indent=2, ensure_ascii=False)
        
        print(f"   ✅ Saved songs database with {len(all_songs)} total songs")
        
        # 2. PLAYLISTS DATABASE - Load existing and merge
        playlists_db_path = self.metadata_output / 'playlists_database.json'
        existing_playlists_db = {'playlists': {}, 'stats': {}}
        
        if playlists_db_path.exists():
            try:
                with open(playlists_db_path, 'r', encoding='utf-8') as f:
                    existing_playlists_db = json.load(f)
                print(f"   📖 Loaded existing playlists database with {len(existing_playlists_db.get('playlists', {}))} playlists")
            except Exception as e:
                print(f"   ⚠️  Warning: Could not load existing playlists database: {e}")
        
        # Merge all playlists (existing + current)
        all_playlists = existing_playlists_db.get('playlists', {}).copy()
        
        # Add/update playlists from current session and add song lists
        for playlist_name, playlist_info in self.playlist_data.items():
            updated_playlist_info = playlist_info.copy()
            
            # Add songs in this playlist
            playlist_songs = []
            for song_id, playlists in self.song_to_playlists.items():
                if playlist_name in playlists:
                    playlist_songs.append(song_id)
            
            updated_playlist_info['songs'] = playlist_songs
            updated_playlist_info['unique_song_count'] = len(playlist_songs)
            all_playlists[playlist_name] = updated_playlist_info
        
        # Save playlists database
        playlists_db = {
            'playlists': all_playlists,
            'stats': {
                'total_playlists': len(all_playlists),
                'generated_at': datetime.now().isoformat(),
                'last_session_stats': self.stats.copy()
            }
        }
        
        with open(playlists_db_path, 'w', encoding='utf-8') as f:
            json.dump(playlists_db, f, indent=2, ensure_ascii=False)
        
        print(f"   ✅ Saved playlists database with {len(all_playlists)} total playlists")
        
        # 3. SONG-PLAYLIST MAPPING - Load existing and merge
        mapping_db_path = self.metadata_output / 'song_playlist_mapping.json'
        existing_mapping_db = {'song_to_playlists': {}, 'stats': {}}
        
        if mapping_db_path.exists():
            try:
                with open(mapping_db_path, 'r', encoding='utf-8') as f:
                    existing_mapping_db = json.load(f)
                print(f"   📖 Loaded existing mapping database with {len(existing_mapping_db.get('song_to_playlists', {}))} mappings")
            except Exception as e:
                print(f"   ⚠️  Warning: Could not load existing mapping database: {e}")
        
        # Merge all mappings (existing + current)
        all_mappings = existing_mapping_db.get('song_to_playlists', {}).copy()
        
        # Add/update mappings from current session
        for song_id, playlists in self.song_to_playlists.items():
            all_mappings[song_id] = playlists.copy()
        
        # Save mapping database
        mapping_db = {
            'song_to_playlists': all_mappings,
            'stats': {
                'total_mappings': len(all_mappings),
                'generated_at': datetime.now().isoformat(),
                'last_session_stats': self.stats.copy()
            }
        }
        
        with open(mapping_db_path, 'w', encoding='utf-8') as f:
            json.dump(mapping_db, f, indent=2, ensure_ascii=False)
        
        print(f"   ✅ Saved mapping database with {len(all_mappings)} total mappings")
        print("✅ All metadata saved successfully!")
    
    def run(self, cleanup_orphans: bool = True):
        """Run the incremental consolidation process"""
        print("🎵 Starting incremental consolidation process...")
        print(f"📁 Base folder: {self.base_songs_folder}")
        print(f"📁 Output folder: {self.output_folder}")
        
        if not self.base_songs_folder.exists():
            print(f"❌ Error: Base songs folder '{self.base_songs_folder}' does not exist!")
            return
        
        # Process each playlist folder
        playlist_folders = [f for f in self.base_songs_folder.iterdir() if f.is_dir()]
        
        if not playlist_folders:
            print("❌ No playlist folders found!")
            return
        
        print(f"📂 Found {len(playlist_folders)} playlist folders")
        
        for playlist_folder in playlist_folders:
            self.process_playlist_folder(playlist_folder)
        
        # Clean up orphaned data if requested
        if cleanup_orphans:
            self.cleanup_orphaned_data()
        
        # Save all metadata
        self.save_metadata()
        
        # Print summary
        print("\n" + "="*60)
        print("🎉 INCREMENTAL CONSOLIDATION COMPLETE")
        print("="*60)
        print(f"🎵 Total unique songs: {len(self.unique_songs)}")
        print(f"📋 Total playlists: {len(self.playlist_data)}")
        print(f"📁 Songs folder: {self.songs_output}")
        print(f"📁 Metadata folder: {self.metadata_output}")
        
        print("\n📊 Session Statistics:")
        print(f"   • Existing songs found: {self.stats['existing_songs_found']}")
        print(f"   • New songs added: {self.stats['new_songs_added']}")
        print(f"   • New playlists added: {self.stats['new_playlists_added']}")
        print(f"   • Playlists updated: {self.stats['playlists_updated']}")
        print(f"   • Duplicates skipped: {self.stats['duplicates_skipped']}")
        print(f"   • Songs added to existing playlists: {self.stats['songs_added_to_existing_playlists']}")
        
        print("\n📄 Generated/Updated files:")
        print("   • songs_database.json (complete song information)")
        print("   • playlists_database.json (playlist information with song lists)")
        print("   • song_playlist_mapping.json (song to playlist relationships)")


def main():
    # Configuration
    SONGS_FOLDER = "songs"  # Your base songs folder
    OUTPUT_FOLDER = "consolidated_music"  # Output folder name
    
    # Create and run consolidator
    consolidator = IncrementalSongConsolidator(SONGS_FOLDER, OUTPUT_FOLDER)
    consolidator.run(cleanup_orphans=True)  # Set to False if you don't want to remove orphaned data


if __name__ == "__main__":
    main()

🔍 Loading existing metadata...
   ✅ Loaded 2152 existing songs
   ✅ Loaded 17 existing playlists
   ✅ Loaded mappings for 2152 songs
📊 Loaded totals:
   • Songs: 2152
   • Playlists: 17
   • Mappings: 2152
✅ Found existing metadata - will merge new data with existing data
🎵 Starting incremental consolidation process...
📁 Base folder: songs
📁 Output folder: consolidated_music
📂 Found 1 playlist folders
🆕 Processing new playlist: Best of tamil songs 2010 2024
   📎 Duplicate found: Omana-Penne-AR-Rahman-Benny-Dayal-Kalyani-Menon.mp3 (existing: song_47b8323eabdb)
   📎 Duplicate found: Yaanji-From-Vikram-Vedha-Sam-CS-Anirudh-Ravichander-Shakthisree-Gopalan.mp3 (existing: song_08d629b4fcbf)
   📎 Duplicate found: Megham-Karukatha-From-Thiruchitrambalam-Dhanush-Anirudh-Ravichander.mp3 (existing: song_08d25ca059f8)
   📎 Duplicate found: Koodamela-Koodavechi-D-Imman-VV-Prassanna-Vandana-Srinivasan.mp3 (existing: song_a33b3161ca83)
   📎 Duplicate found: Unakaga-AR-Rahman-Sreekanth-Hariharan-Madhu